In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Import Libraries

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Data Preprocessing

### Training Image Preprocessing

In [2]:
training_set = tf.keras.utils.image_dataset_from_directory(
    "train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 70295 files belonging to 38 classes.


### Validation Image Preprocessing

In [3]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    "valid",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 17572 files belonging to 38 classes.


In [4]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

In [5]:
for x, y in training_set:
    print(x, x.shape)
    print(y, y.shape)
    break

tf.Tensor(
[[[[147.5  145.5  146.5 ]
   [148.25 146.25 147.25]
   [148.   146.   147.  ]
   ...
   [156.5  152.5  149.5 ]
   [156.5  152.5  149.5 ]
   [154.5  150.5  147.5 ]]

  [[153.75 151.75 152.75]
   [156.75 154.75 155.75]
   [149.   147.   148.  ]
   ...
   [156.5  152.5  149.5 ]
   [156.5  152.5  149.5 ]
   [155.75 151.75 148.75]]

  [[146.25 144.25 145.25]
   [149.25 147.25 148.25]
   [145.25 143.25 144.25]
   ...
   [156.5  152.5  149.5 ]
   [156.5  152.5  149.5 ]
   [157.25 153.25 150.25]]

  ...

  [[ 65.25  62.25  57.25]
   [ 66.5   63.5   58.5 ]
   [ 66.5   63.5   58.5 ]
   ...
   [109.   105.   102.  ]
   [110.   106.   103.  ]
   [112.5  108.5  105.5 ]]

  [[ 64.75  61.75  56.75]
   [ 65.75  62.75  57.75]
   [ 65.75  62.75  57.75]
   ...
   [108.25 104.25 101.25]
   [109.   105.   102.  ]
   [106.75 102.75  99.75]]

  [[ 64.    61.    56.  ]
   [ 65.    62.    57.  ]
   [ 65.    62.    57.  ]
   ...
   [103.5   99.5   96.5 ]
   [101.5   97.5   94.5 ]
   [ 98.5   94.5   9

### To avoid overshooting
1. Choose small learing rate. Default is 0.001, we are taking 0.0001
2. There may be a chance of under fitting, so increase the number of neurons.
3. Add more convolution layers to extract more features.

## Building Model

In [6]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential

In [7]:
model = Sequential()

## Convolutional Layer

In [8]:
model.add(Conv2D(filters=32,kernel_size=3,padding="same",activation="relu",input_shape=[128,128,3]))
model.add(Conv2D(filters=32,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2,strides=2))

In [9]:
model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"))
model.add(Conv2D(filters=64,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2,strides=2))

In [10]:
model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"))
model.add(Conv2D(filters=128,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2,strides=2))

In [11]:
model.add(Conv2D(filters=256,kernel_size=3,padding="same",activation="relu"))
model.add(Conv2D(filters=256,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2,strides=2))

In [12]:
model.add(Conv2D(filters=512,kernel_size=3,padding="same",activation="relu"))
model.add(Conv2D(filters=512,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2,strides=2))

In [13]:
model.add(Dropout(0.25)) # To avoid overfitting

In [14]:
model.add(Flatten())

In [15]:
model.add(Dense(units=1500,activation="relu"))

In [16]:
model.add(Dropout(0.4))

In [17]:
#Output Layer
model.add(Dense(units=38,activation="softmax"))

## Compiling Model

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001),loss="categorical_crossentropy",metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                    

## Model Training

In [20]:
training_history = model.fit(x=training_set,validation_data=validation_set,epochs=10)

Epoch 1/10
2197/2197 [==============================] - 1784s 811ms/step - loss: 1.3781 - accuracy: 0.5954 - val_loss: 0.4970 - val_accuracy: 0.8422
Epoch 2/10
2197/2197 [==============================] - 1749s 796ms/step - loss: 0.4597 - accuracy: 0.8554 - val_loss: 0.3510 - val_accuracy: 0.8866
Epoch 3/10
2197/2197 [==============================] - 1757s 799ms/step - loss: 0.2757 - accuracy: 0.9118 - val_loss: 0.2133 - val_accuracy: 0.9309
Epoch 4/10
2197/2197 [==============================] - 1763s 803ms/step - loss: 0.1914 - accuracy: 0.9374 - val_loss: 0.1726 - val_accuracy: 0.9452
Epoch 5/10
2197/2197 [==============================] - 1755s 799ms/step - loss: 0.1367 - accuracy: 0.9549 - val_loss: 0.1467 - val_accuracy: 0.9520
Epoch 6/10
2197/2197 [==============================] - 1747s 795ms/step - loss: 0.1064 - accuracy: 0.9654 - val_loss: 0.1724 - val_accuracy: 0.9440
Epoch 7/10
2197/2197 [==============================] - 1751s 797ms/step - loss: 0.0943 - accuracy: 0.9690

In [23]:
#storing trainging_history in ipython store
#%store training_history
#%store

## Model Evaluation

In [24]:
# Model Evaludation on traning set
train_loss, train_acc = model.evaluate(training_set)

2197/2197 [==============================] - 519s 236ms/step - loss: 0.0202 - accuracy: 0.9937


In [25]:
print(train_loss, train_acc)

0.020162394270300865 0.993698000907898


In [26]:
# Model Evaluation on validation set
val_loss, val_acc = model.evaluate(validation_set)

550/550 [==============================] - 133s 242ms/step - loss: 0.1070 - accuracy: 0.9682


In [27]:
print(val_loss, val_acc)

0.10699165612459183 0.9682449102401733


## Saving Model

In [30]:
model.save("trained_model.keras")

In [31]:
training_history.history

{'loss': [1.378056287765503,
  0.45971667766571045,
  0.2757202982902527,
  0.19135701656341553,
  0.13666775822639465,
  0.10638018697500229,
  0.09434140473604202,
  0.07309447973966599,
  0.06342809647321701,
  0.057877566665410995],
 'accuracy': [0.5953766107559204,
  0.8553666472434998,
  0.9117575883865356,
  0.9374350905418396,
  0.954904317855835,
  0.9654029607772827,
  0.9690447449684143,
  0.9761860966682434,
  0.9797425270080566,
  0.9812504649162292],
 'val_loss': [0.4969624876976013,
  0.350989431142807,
  0.21327729523181915,
  0.17257751524448395,
  0.1466531753540039,
  0.17239238321781158,
  0.13537299633026123,
  0.12431935966014862,
  0.15972383320331573,
  0.10699170082807541],
 'val_accuracy': [0.8421921133995056,
  0.886637806892395,
  0.9309127926826477,
  0.9451969265937805,
  0.9520259499549866,
  0.9440017938613892,
  0.9583997130393982,
  0.9637491703033447,
  0.9544730186462402,
  0.9682449102401733]}

In [32]:
# Recording History in JSON
import json
with open("training_hist.json", "w") as f:
    json.dump(training_history.history,f)

## Accuracy Visualization

In [ ]:
epochs = [i for i in range(1,11)]
plt.plot(epochs, training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs, training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel("No of Epochs")
plt.ylabel("Accuracy Result")
plt.title("Visualization of Accuracy Result")
plt.legend()
plt.show()

In [ ]:
class_name = validation_set.class_names
class_name

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    "valid",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
predicted_categories = tf.argmax(y_pred,axis=1)

In [ ]:
true_categories = tf.concat([y for x, y in test_set],axis=0)

In [ ]:
Y_true = tf.argmax(true_categories,axis=1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(Y_true, predicted_categories, target_names=class_name))

In [ ]:
cm = confusion_matrix(Y_true, predicted_categories)

## Confusion Matrix visualization

In [ ]:
plt.figure(figsize=(40,40))
sns.heatmap(cm,annot=True,annot_kws={'size':10})
plt.xlabel("Predicted Class",fontsize=20)
plt.ylabel("Actual Class",fontsize=20)
plt.title("Plant Disease Prediction Confusion Matrix",fontsize=25)
plt.show()